# Text Summarization and Document Based Question Answering System with OpenAI's GPT-3 using LangChain.

## Relevant Libraries

In [1]:
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import ast
from langchain.chains import RetrievalQA

In [2]:
# Get API key
with open('api_key.txt') as f:
    api_key = f.readline()

## Load data into an LLM compatible format

### Load Pdf data with Langchain

In [3]:
loader = PyPDFLoader("pdf_data/Big Data Applications in Industry.pdf")
pages = loader.load()

In [4]:
len(pages)

89

In [5]:
page = pages[0]

In [6]:
# Print the first 100 characters
print(page.page_content[0:100])

See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.


In [7]:
# Print entire page content
print(page)

page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/357264007\nRole of Big Data Analytics in the Financial Service Sector\nChapt er · Dec ember 2021\nDOI: 10.1201/9781003175889-9\nCITATIONS\n0READS\n1,342\n2 author s, including:\nSome o f the author s of this public ation ar e also w orking on these r elat ed pr ojects:\nhuman r esour ce de velopment  View pr oject\nENTREPRENEUR SHIP  View pr oject\nRamanujam V eluchamy\nBhar athiar Univ ersity\n98 PUBLICA TIONS \xa0\xa0\xa083 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Ramanujam V eluchamy  on 07 F ebruar y 2022.\nThe user has r equest ed enhanc ement of the do wnlo aded file.' metadata={'source': 'pdf_data/Big Data Applications in Industry.pdf', 'page': 0}


In [8]:
# Print page metadata
page.metadata

{'source': 'pdf_data/Big Data Applications in Industry.pdf', 'page': 0}

### Load url data with Langchain

In [9]:
# Use the WebBaseLoader class to load the URL data
# Create an instance of the WebBaseLoader class with the relevant URL
loader = WebBaseLoader("https://www.simplilearn.com/tutorials/big-data-tutorial/big-data-applications")

In [10]:
docs = loader.load()

In [11]:
# print first 500 characters
print(docs[0].page_content[:500])








14 Big Data Examples & Applications Across Industries





















































All CoursesAll Courses Log inBig DataData Science & Business AnalyticsAI & Machine LearningProject ManagementCyber SecurityCloud ComputingDevOpsBusiness and LeadershipQuality ManagementSoftware DevelopmentAgile and ScrumIT Service and ArchitectureDigital MarketingBig DataCareer Fast-trackEnterpriseOther SegmentsTutorialsArticlesEbooksFree Practice TestsOn-demand WebinarsLive Webinars


### Load data resources

In [12]:
# Create a list of loaders to load different types of resources(pdf, url)
loaders = [
    # Load a PDF document using PyPDFLoader
    PyPDFLoader("pdf_data/Big Data Applications in Industry.pdf"),
    
    # Load content from a URL using WebBaseLoader
    WebBaseLoader("https://www.simplilearn.com/tutorials/big-data-tutorial/big-data-applications")
]

# Create an empty list to store the loaded documents
docs = []

# Iterate through the loaders and load documents into the 'docs' list
for loader in loaders:
    # Use the 'load' method of each loader to load documents and extend the 'docs' list
    docs.extend(loader.load())

## Split Document

We split the texts/documents to facilitate the retrieval process, allowing it to retrieve only the relevant chunks based on the search parameters.

Searching through, for example, a 1000-page document for a topic relevant to only 3 pages is counterproductive.

This is why splitting is necessary to speed up the process and enhance the overall performance of the model while finding relationships between the relevant splits/chunks and using them to answer user prompts.

Below is a visual representation of how Langchain accomplishes this.

In [13]:
# Split the documents using the RecursiveCharacterTextSplitter
# Specify the parameters for the split
text_splitter = RecursiveCharacterTextSplitter(
    # Define the size of each text chunk
    chunk_size=1500,
    
    # Define the overlap between consecutive text chunks
    chunk_overlap=150
)

In [14]:
splits = text_splitter.split_documents(docs)

In [15]:
# Number of splits
len(splits)

227

In [16]:
# splits

## Create Embeddings

In [17]:
# Create text embeddings using the OpenAIEmbeddings class
embedding = OpenAIEmbeddings(openai_api_key=api_key)

In [18]:
# Use the Chroma embedding vector database, to store the embeddings vectors
# Define a permanent directory for the embeddings vectors
persist_directory = 'docs/chroma/'

In [19]:
# # Create a Chroma vector store from a list of documents using the provided embedding
# vectordb = Chroma.from_documents(
#     # List of documents
#     documents=splits,
    
#     # Embeddings model
#     embedding=embedding,
    
#     # Directory for vector storage
#     persist_directory=persist_directory  
# )

# Create a Chroma vector store with the specified persistence directory and embedding function
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [20]:
# Number of vectors in the vector store
print(vectordb._collection.count())

227


## Question Answering Retrieval Chain

In [21]:
# Create an instance of the ChatOpenAI class with specified parameters
llm = ChatOpenAI(
    # model name/version
    model_name="gpt-3.5-turbo",
    
    # API key
    openai_api_key=api_key,
    
    # Set the temperature parameter to control response variability.
    # Note Low variability gives high fidelity in answering questions.
    temperature=0
)

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

## Summarize text with GPT 3 using LangChain

In [23]:
question = f"""
Your task is to briefly provide an overall summary of the data,
like a professional research assistant."""

# Performs a similarity search in the vector database (vectordb) to retrieve relevant splits.
docs = vectordb.similarity_search(question)
len(docs)

4

In [24]:
# Use the Question Answering (QA) chain (qa_chain) to query and obtain a result.
result = qa_chain({"query": question})

In [25]:
# Extract and return the response from the result dictionary.
result["result"]

'This book provides a comprehensive guide on Big Data, its tools, and applications. It emphasizes the importance of involving industry experts in the development of curriculum and highlights the role of research in driving innovation. The book covers various topics such as data integration, forecasting principles and models, the application of Big Data in diagnosing diabetes, and the use of data mining techniques for satellite performance analysis and text mining in biomedicine and healthcare. It also offers guidance on how to use the book based on different roles, including students, software developers, teachers, members of the Board of Studies, and researchers. Overall, this book serves as a valuable resource for anyone interested in gaining a basic understanding of Big Data and its practical applications.'

## Document Based Question Answering System

Chat with you data.

### Prompt Function

In [26]:
# Use LangChains' ConversationBufferMemory function to store conversations.
memory = ConversationBufferMemory()

In [27]:
def get_user_input():
    user_input = []  # Initialize an empty list to store user input
    
    while True:
        # Continuously ask for user input
        print("\033[1m" + "Ask a question ? (or type 'quit' to exit): " + "\033[0m")
        # text = input("Ask a question ? (or type 'quit' to exit): ")
        text = input()
        print()
        
        # Check if the user wants to quit
        if text.lower() == 'quit':
            break  # Exit the loop if the user enters 'quit'
        
        # Get the response from openai's api using the qa_chain function    
        else:
            chatgpt_response = qa_chain({"query": text})
            print("\033[1m" + "ChatBot:\n" + "\033[0m" + f" {chatgpt_response['result']} ")
            print()
        
        # Save the conversation using LangChains ConversationBufferMemory function
        memory.save_context({"input": text}, {"output": chatgpt_response['result']})

    return user_input

In [28]:
user_responses = get_user_input()

Ask a question ? (or type 'quit' to exit): 


 What are five specific industries that have successfully adopted big data analytics, and what are the prominent use cases within each of these industries?



ChatBot:
 Five specific industries that have successfully adopted big data analytics are:

1. Banking and Securities:
- Securities fraud early warning: Using big data analytics to detect and prevent fraudulent trading activities in financial markets.
- Trade visibility: Analyzing large volumes of data to gain insights into trade patterns and market trends.
- Customer data transformation: Utilizing big data analytics to transform customer data into actionable insights for personalized marketing and customer service.

2. Government:
- Energy exploration: Analyzing big data to identify potential energy sources and optimize energy production.
- Fraud detection: Using big data analytics to detect and prevent fraudulent activities in public services, such as social disability claims.
- Environmental protection: Leveraging big data to monitor and analyze environmental data for better decision-making and resource management.

3. Retail:
- Demand forecasting: Utilizing big data analytics to pr

 How does the utilization of big data analytics impact the efficiency, productivity, and competitiveness of companies within various industrial sectors?



ChatBot:
 The utilization of big data analytics has a significant impact on the efficiency, productivity, and competitiveness of companies across various industrial sectors. Here are some specific ways in which it affects these areas:

1. Efficiency: Big data analytics enables companies to streamline their operations and make data-driven decisions. By analyzing large volumes of data, companies can identify inefficiencies in their processes, optimize resource allocation, and reduce waste. This leads to improved operational efficiency and cost savings.

2. Productivity: Big data analytics provides valuable insights that can enhance productivity. By analyzing data on employee performance, companies can identify areas for improvement and implement targeted training programs. Additionally, predictive analytics can help optimize production schedules, inventory management, and supply chain operations, leading to increased productivity.

3. Competitiveness: Big data analytics gives companies 

 What are the ethical and legal considerations associated with the use of big data in industrial applications, and how are organizations addressing issues related to data privacy, security, and compliance?



ChatBot:
 The provided context does not specifically mention the ethical and legal considerations associated with the use of big data in industrial applications or how organizations are addressing issues related to data privacy, security, and compliance. Therefore, I don't have enough information to answer your question accurately. 

Ask a question ? (or type 'quit' to exit): 


 quit


### Display conversation history

In [29]:
# Load memory
memory.load_memory_variables({})

{'history': "Human: What are five specific industries that have successfully adopted big data analytics, and what are the prominent use cases within each of these industries?\nAI: Five specific industries that have successfully adopted big data analytics are:\n\n1. Banking and Securities:\n- Securities fraud early warning: Using big data analytics to detect and prevent fraudulent trading activities in financial markets.\n- Trade visibility: Analyzing large volumes of data to gain insights into trade patterns and market trends.\n- Customer data transformation: Utilizing big data analytics to transform customer data into actionable insights for personalized marketing and customer service.\n\n2. Government:\n- Energy exploration: Analyzing big data to identify potential energy sources and optimize energy production.\n- Fraud detection: Using big data analytics to detect and prevent fraudulent activities in public services, such as social disability claims.\n- Environmental protection: Lev

In [30]:
# Load memory
memory.load_memory_variables({})

# Convert the conversation history dictionary to a well-formatted output

input_text = str(memory.load_memory_variables({}))

# Parse the input text as a dictionary
history = ast.literal_eval(input_text)

# Extract the conversation
conversation = history['history']

# Split the conversation into lines
lines = conversation.split('\n')

# Iterate through the lines and format the conversation
count = 0
for line in lines:
    if line.startswith('Human'):
        print()
        print()
        print( "\033[1m" + line + "\033[0m")
        print()
        
    else:
        print(line)
        
    count += 1



Human: What are five specific industries that have successfully adopted big data analytics, and what are the prominent use cases within each of these industries?

AI: Five specific industries that have successfully adopted big data analytics are:

1. Banking and Securities:
- Securities fraud early warning: Using big data analytics to detect and prevent fraudulent trading activities in financial markets.
- Trade visibility: Analyzing large volumes of data to gain insights into trade patterns and market trends.
- Customer data transformation: Utilizing big data analytics to transform customer data into actionable insights for personalized marketing and customer service.

2. Government:
- Energy exploration: Analyzing big data to identify potential energy sources and optimize energy production.
- Fraud detection: Using big data analytics to detect and prevent fraudulent activities in public services, such as social disability claims.
- Environmental protection: Leveraging big data to m